### Importing libraries

In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import Sequential
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow_hub as hub
import numpy as np
from google.colab import drive
drive.mount("/content/drive")
os.chdir("/content/drive/MyDrive/MachineLearningProject")

Mounted at /content/drive


### Loading data set

In [2]:
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory("/content/drive/MyDrive/MachineLearningProject/data/train",
                                                               label_mode="binary",
                                                               color_mode="rgb",
                                                               batch_size=32,
                                                               image_size=(64, 64),
                                                               shuffle=True,
                                                               seed=42,
                                                               subset="both",
                                                               validation_split=0.25)

test_ds = tf.keras.utils.image_dataset_from_directory("/content/drive/MyDrive/MachineLearningProject/data/test",
                                                      label_mode="binary",
                                                      color_mode="rgb",
                                                      batch_size=32,
                                                      image_size=(64, 64),
                                                      shuffle=True,
                                                      seed=42)

Found 4743 files belonging to 2 classes.
Using 3558 files for training.
Using 1185 files for validation.
Found 1184 files belonging to 2 classes.


## Data Preprocessing Pipeline

In [7]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(mode="horizontal"),
    tf.keras.layers.RandomContrast(factor=0.3),
    tf.keras.layers.Lambda(function=tf.image.per_image_standardization)
])

In [4]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Activation
from tensorflow.keras.optimizers import Adam


In [12]:
model_1 = Sequential([
    tf.keras.layers.RandomFlip(mode="horizontal", input_shape=(64, 64, 3)),
    tf.keras.layers.RandomContrast(factor=0.3),
    tf.keras.layers.Lambda(function=tf.image.per_image_standardization),

    Conv2D(filters=10,
           kernel_size=(3, 3),
           strides=(1, 1),
           padding='valid',
           activation="relu",
           kernel_initializer="glorot_normal"),

    MaxPool2D(pool_size=(2, 2),
              strides=None,
              padding="valid"),

    Conv2D(filters=10,
           kernel_size=(3, 3),
           strides=(1, 1),
           padding='valid',
           activation="relu",
           kernel_initializer="glorot_normal"),

    MaxPool2D(pool_size=(2, 2),
              strides=None,
              padding="valid"),

    Conv2D(filters=10,
           kernel_size=(3, 3),
           strides=(1, 1),
           padding='valid',
           activation="relu",
           kernel_initializer="glorot_normal"),

    MaxPool2D(pool_size=(2, 2),
              strides=None,
              padding="valid"),

    Flatten(),

    Dense(1,
          activation="sigmoid",
          kernel_initializer="glorot_normal",
          bias_initializer="zeros")
])

In [13]:
model_1.compile(loss="binary_crossentropy",
                optimizer=Adam(),
                metrics=["accuracy"])

print(model_1.summary())

history_1 = model_1.fit(train_ds,
                        epochs=50,
                        steps_per_epoch=len(train_ds),
                        validation_data=test_ds,
                        validation_steps=len(test_ds))

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip_4 (RandomFlip)  (None, 64, 64, 3)         0         
                                                                 
 random_contrast_4 (RandomC  (None, 64, 64, 3)         0         
 ontrast)                                                        
                                                                 
 lambda_4 (Lambda)           (None, 64, 64, 3)         0         
                                                                 
 conv2d_9 (Conv2D)           (None, 62, 62, 10)        280       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 31, 31, 10)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 29, 29, 10)       

In [14]:
model_2 = Sequential([
    tf.keras.layers.RandomFlip(mode="horizontal", input_shape=(64, 64, 3)),
    tf.keras.layers.RandomContrast(factor=0.3),
    tf.keras.layers.Lambda(function=tf.image.per_image_standardization),

    Conv2D(filters=10,
           kernel_size=(3, 3),
           strides=(1, 1),
           padding='valid',
           activation="relu",
           kernel_initializer="glorot_normal"),

    MaxPool2D(pool_size=(2, 2),
              strides=None,
              padding="valid"),

    tf.keras.layers.BatchNormalization(),

    Conv2D(filters=10,
           kernel_size=(3, 3),
           strides=(1, 1),
           padding='valid',
           activation="relu",
           kernel_initializer="glorot_normal"),

    MaxPool2D(pool_size=(2, 2),
              strides=None,
              padding="valid"),

    tf.keras.layers.BatchNormalization(),

    Conv2D(filters=10,
           kernel_size=(3, 3),
           strides=(1, 1),
           padding='valid',
           activation="relu",
           kernel_initializer="glorot_normal"),

    MaxPool2D(pool_size=(2, 2),
              strides=None,
              padding="valid"),

    tf.keras.layers.BatchNormalization(),

    Flatten(),

    Dense(1,
          activation="sigmoid",
          kernel_initializer="glorot_normal",
          bias_initializer="zeros")
])

In [16]:
early_stopper = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, verbose=1)

In [17]:
model_2.compile(loss="binary_crossentropy",
                optimizer=Adam(),
                metrics=["accuracy"])

print(model_2.summary())

history_2 = model_2.fit(train_ds,
                        epochs=50,
                        steps_per_epoch=len(train_ds),
                        validation_data=test_ds,
                        validation_steps=len(test_ds),
                        callbacks=[early_stopper])

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip_5 (RandomFlip)  (None, 64, 64, 3)         0         
                                                                 
 random_contrast_5 (RandomC  (None, 64, 64, 3)         0         
 ontrast)                                                        
                                                                 
 lambda_5 (Lambda)           (None, 64, 64, 3)         0         
                                                                 
 conv2d_12 (Conv2D)          (None, 62, 62, 10)        280       
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 31, 31, 10)        0         
 ng2D)                                                           
                                                                 
 batch_normalization (Batch  (None, 31, 31, 10)       

In [20]:
model_3 = Sequential([
    tf.keras.layers.RandomFlip(mode="horizontal", input_shape=(256, 256, 3)),
    tf.keras.layers.RandomContrast(factor=0.3),
    tf.keras.layers.Lambda(function=tf.image.per_image_standardization),

    Conv2D(filters=10,
           kernel_size=(3, 3),
           strides=(1, 1),
           padding='valid',
           activation="relu",
           kernel_initializer="glorot_normal"),

    MaxPool2D(pool_size=(2, 2),
              strides=None,
              padding="valid"),

    tf.keras.layers.BatchNormalization(),

    Conv2D(filters=10,
           kernel_size=(3, 3),
           strides=(1, 1),
           padding='valid',
           activation="relu",
           kernel_initializer="glorot_normal"),

    MaxPool2D(pool_size=(2, 2),
              strides=None,
              padding="valid"),

    tf.keras.layers.BatchNormalization(),

    Conv2D(filters=10,
           kernel_size=(3, 3),
           strides=(1, 1),
           padding='valid',
           activation="relu",
           kernel_initializer="glorot_normal"),

    MaxPool2D(pool_size=(2, 2),
              strides=None,
              padding="valid"),

    tf.keras.layers.BatchNormalization(),

        Conv2D(filters=10,
           kernel_size=(3, 3),
           strides=(1, 1),
           padding='valid',
           activation="relu",
           kernel_initializer="glorot_normal"),

    MaxPool2D(pool_size=(2, 2),
              strides=None,
              padding="valid"),

    tf.keras.layers.BatchNormalization(),

        Conv2D(filters=10,
           kernel_size=(3, 3),
           strides=(1, 1),
           padding='valid',
           activation="relu",
           kernel_initializer="glorot_normal"),

    MaxPool2D(pool_size=(2, 2),
              strides=None,
              padding="valid"),

    tf.keras.layers.BatchNormalization(),

        Conv2D(filters=10,
           kernel_size=(3, 3),
           strides=(1, 1),
           padding='valid',
           activation="relu",
           kernel_initializer="glorot_normal"),

    MaxPool2D(pool_size=(2, 2),
              strides=None,
              padding="valid"),

    tf.keras.layers.BatchNormalization(),

    Flatten(),

    Dense(1,
          activation="sigmoid",
          kernel_initializer="glorot_normal",
          bias_initializer="zeros")
])

In [21]:
train_ds = tf.keras.utils.image_dataset_from_directory("/content/drive/MyDrive/MachineLearningProject/data/train",
                                                               label_mode="binary",
                                                               color_mode="rgb",
                                                               batch_size=32,
                                                               image_size=(256, 256),
                                                               shuffle=True)

test_ds = tf.keras.utils.image_dataset_from_directory("/content/drive/MyDrive/MachineLearningProject/data/test",
                                                      label_mode="binary",
                                                      color_mode="rgb",
                                                      batch_size=32,
                                                      image_size=(256, 256),
                                                      shuffle=True,
                                                      seed=42)

Found 4743 files belonging to 2 classes.
Found 1184 files belonging to 2 classes.


In [23]:
model_3.compile(loss="binary_crossentropy",
                optimizer=Adam(),
                metrics=["accuracy"])

print(model_3.summary())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip_8 (RandomFlip)  (None, 256, 256, 3)       0         
                                                                 
 random_contrast_8 (RandomC  (None, 256, 256, 3)       0         
 ontrast)                                                        
                                                                 
 lambda_8 (Lambda)           (None, 256, 256, 3)       0         
                                                                 
 conv2d_27 (Conv2D)          (None, 254, 254, 10)      280       
                                                                 
 max_pooling2d_27 (MaxPooli  (None, 127, 127, 10)      0         
 ng2D)                                                           
                                                                 
 batch_normalization_15 (Ba  (None, 127, 127, 10)     

In [ ]:
history_2 = model_2.fit(train_ds,
                        epochs=50,
                        steps_per_epoch=len(train_ds),
                        validation_data=test_ds,
                        validation_steps=len(test_ds),
                        callbacks=[early_stopper])

### Data preprocessing

In [ ]:
image_batch, labels_batch = next(iter(train_ds))
first_image = image_batch[0]
first_image

<tf.Tensor: shape=(64, 64, 3), dtype=float32, numpy=
array([[[128.8125  ,  77.6875  , 126.6875  ],
        [130.0625  ,  72.77344 , 112.66406 ],
        [147.89844 ,  94.77344 , 133.64844 ],
        ...,
        [118.125   ,  71.875   , 109.53906 ],
        [138.85156 ,  61.085938, 100.96094 ],
        [112.375   , 121.125   ,  97.75    ]],

       [[139.77344 , 105.83594 , 117.14844 ],
        [117.25    ,  81.3125  , 101.625   ],
        [146.76562 , 120.25    , 132.75781 ],
        ...,
        [142.3125  ,  50.6875  ,  94.      ],
        [129.3125  ,  41.3125  ,  89.3125  ],
        [103.875   ,  40.      ,  71.125   ]],

       [[ 86.08594 ,  54.171875,  69.609375],
        [ 63.875   ,  29.195312,  46.429688],
        [ 79.375   ,  32.375   ,  52.375   ],
        ...,
        [118.92969 , 123.80469 , 112.17969 ],
        [107.88281 , 105.61719 , 102.4375  ],
        [ 66.16406 ,  71.765625,  69.85156 ]],

       ...,

       [[187.67188 , 128.      ,  75.671875],
        [ 58.87

In [ ]:
new_image = tf.image.per_image_standardization(first_image)
new_image

<tf.Tensor: shape=(64, 64, 3), dtype=float32, numpy=
array([[[ 0.07427142, -0.666327  ,  0.0434886 ],
        [ 0.09237896, -0.7375123 , -0.15965538],
        [ 0.35075092, -0.41881955,  0.14432497],
        ...,
        [-0.08054806, -0.7505271 , -0.20492423],
        [ 0.21969761, -0.9068178 , -0.3291872 ],
        [-0.16384274, -0.03708996, -0.37570098]],

       [[ 0.23305193, -0.2585678 , -0.09469457],
        [-0.09322333, -0.6138151 , -0.3195676 ],
        [ 0.334341  , -0.04976523,  0.13142335],
        ...,
        [ 0.26983285, -1.0574499 , -0.43002358],
        [ 0.08151444, -1.1932565 , -0.49792686],
        [-0.286974  , -1.2122693 , -0.7613916 ]],

       [[-0.54466695, -1.0069752 , -0.783347  ],
        [-0.8664153 , -1.3687865 , -1.1191287 ],
        [-0.6418818 , -1.3227254 , -1.0330048 ],
        ...,
        [-0.06889132,  0.00172809, -0.16667205],
        [-0.22891672, -0.26173663, -0.3077977 ],
        [-0.8332559 , -0.7521115 , -0.7798386 ]],

       ...,

       

In [ ]:
#img_new = first_image.map(lambda x, y: (tf.keras.applications.resnet50.preprocess_input(x), y))
img_new = tf.keras.applications.resnet50.preprocess_input(first_image)
img_new

<tf.Tensor: shape=(64, 64, 3), dtype=float32, numpy=
array([[[ 37.060997 ,  37.221    ,  46.32     ],
        [ 40.342247 ,  41.221    ,  50.32     ],
        [ 41.060997 ,  49.221    ,  57.32     ],
        ...,
        [-15.939003 ,  29.50225  ,  50.7575   ],
        [-12.579628 ,  28.580376 ,  44.679375 ],
        [-10.545937 ,  30.614067 ,  46.713066 ]],

       [[ 36.060997 ,  39.2918   ,  45.264824 ],
        [ 39.107872 ,  42.267876 ,  48.835625 ],
        [ 37.40963  ,  46.330376 ,  54.429375 ],
        ...,
        [ -2.913124 ,  26.580376 ,  39.679375 ],
        [ 15.476524 ,  41.921196 ,  48.26043  ],
        [ 29.311974 ,  53.471977 ,  53.945976 ]],

       [[ 31.73629  ,  37.896294 ,  45.995293 ],
        [ 36.680626 ,  42.84063  ,  50.93963  ],
        [ 36.326622 ,  46.486626 ,  54.585625 ],
        ...,
        [ 60.595177 ,  72.7835   ,  65.01043  ],
        [ 67.72213  ,  83.455376 ,  73.554375 ],
        [ 55.823204 ,  70.88946  ,  64.39471  ]],

       ...,

       

### Resnet

In [ ]:
resnet_model = Sequential([
    tf.keras.layers.Lambda(preprocess_input, input_shape=(64, 64, 3)),
    ResNet50(include_top=False, pooling="avg", weights="imagenet", input_shape=(64, 64, 3)),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

resnet_model.layers[1].trainable = False

94765736/94765736 [==============================] - 3s 0us/step


In [ ]:
resnet_v2 = tf.keras.Sequential([
    tf.keras.layers.Lambda(tf.keras.applications.resnet_v2.preprocess_input, input_shape=(64, 64, 3)),
    tf.keras.applications.resnet_v2.ResNet50V2(include_top=False, weights="imagenet", input_shape=(64, 64, 3),  pooling="avg"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

resnet_v2.layers[1].trainable = False

In [ ]:
resnet_v2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_2 (Lambda)           (None, 64, 64, 3)         0         
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 dense_2 (Dense)             (None, 1)                 2049      
                                                                 
Total params: 23566849 (89.90 MB)
Trainable params: 2049 (8.00 KB)
Non-trainable params: 23564800 (89.89 MB)
_________________________________________________________________


In [ ]:
resnet_model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_9 (Lambda)           (None, 64, 64, 3)         0         
                                                                 
 keras_layer_6 (KerasLayer)  (None, 2048)              23564800  
                                                                 
 dense_9 (Dense)             (None, 1)                 2049      
                                                                 
Total params: 23566849 (89.90 MB)
Trainable params: 2049 (8.00 KB)
Non-trainable params: 23564800 (89.89 MB)
_________________________________________________________________


In [ ]:
resnet_v2.compile(optimizer="Adam",
                     loss="binary_crossentropy",
                     metrics=["accuracy"])

In [ ]:
resnet_v2.fit(train_ds,
                 epochs=10,
                 steps_per_epoch=len(train_ds),
                 validation_data=test_ds,
                 validation_steps=len(test_ds))

Epoch 1/10
112/112 [==============================] - 26s 176ms/step - loss: 0.3600 - accuracy: 0.8648 - val_loss: 0.2706 - val_accuracy: 0.8953
Epoch 2/10
112/112 [==============================] - 20s 168ms/step - loss: 0.2008 - accuracy: 0.9295 - val_loss: 0.2486 - val_accuracy: 0.9062
Epoch 3/10
112/112 [==============================] - 20s 163ms/step - loss: 0.1614 - accuracy: 0.9382 - val_loss: 0.2360 - val_accuracy: 0.9088
Epoch 4/10
112/112 [==============================] - 19s 162ms/step - loss: 0.1340 - accuracy: 0.9519 - val_loss: 0.2320 - val_accuracy: 0.9155
Epoch 5/10
112/112 [==============================] - 21s 177ms/step - loss: 0.1148 - accuracy: 0.9584 - val_loss: 0.2335 - val_accuracy: 0.9164
Epoch 6/10
112/112 [==============================] - 19s 163ms/step - loss: 0.1028 - accuracy: 0.9666 - val_loss: 0.2322 - val_accuracy: 0.9181
Epoch 7/10
112/112 [==============================] - 20s 168ms/step - loss: 0.0929 - accuracy: 0.9663 - val_loss: 0.2372 - val_ac

In [ ]:
resnet_v2.evaluate(test_ds)

37/37 [==============================] - 5s 99ms/step - loss: 0.2433 - accuracy: 0.9155


[0.24325056374073029, 0.9155405163764954]

### Efficientnet B0

In [ ]:
efficientnetb0 = tf.keras.Sequential([
    tf.keras.layers.Lambda(tf.keras.applications.efficientnet.preprocess_input, input_shape=(64, 64, 3)),
    tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, weights="imagenet", input_shape=(64, 64, 3),  pooling="avg"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

efficientnetb0.layers[1].trainable = False

16705208/16705208 [==============================] - 0s 0us/step


In [ ]:
efficientnetb0.compile(optimizer="Adam",
                     loss="binary_crossentropy",
                     metrics=["accuracy"])

In [ ]:
efficientnetb0.fit(train_ds,
                 epochs=10,
                 steps_per_epoch=len(train_ds),
                 validation_data=test_ds,
                 validation_steps=len(test_ds))

Epoch 1/10
112/112 [==============================] - 29s 189ms/step - loss: 0.3504 - accuracy: 0.8626 - val_loss: 0.2155 - val_accuracy: 0.9181
Epoch 2/10
112/112 [==============================] - 19s 166ms/step - loss: 0.1883 - accuracy: 0.9365 - val_loss: 0.1708 - val_accuracy: 0.9299
Epoch 3/10
112/112 [==============================] - 19s 157ms/step - loss: 0.1600 - accuracy: 0.9435 - val_loss: 0.1562 - val_accuracy: 0.9316
Epoch 4/10
112/112 [==============================] - 19s 158ms/step - loss: 0.1460 - accuracy: 0.9486 - val_loss: 0.1471 - val_accuracy: 0.9333
Epoch 5/10
112/112 [==============================] - 20s 169ms/step - loss: 0.1310 - accuracy: 0.9542 - val_loss: 0.1418 - val_accuracy: 0.9350
Epoch 6/10
112/112 [==============================] - 24s 211ms/step - loss: 0.1233 - accuracy: 0.9581 - val_loss: 0.1387 - val_accuracy: 0.9316
Epoch 7/10
112/112 [==============================] - 19s 160ms/step - loss: 0.1181 - accuracy: 0.9604 - val_loss: 0.1352 - val_ac

In [ ]:
efficientnetb0.evaluate(test_ds)

37/37 [==============================] - 5s 98ms/step - loss: 0.1303 - accuracy: 0.9392


[0.1302599012851715, 0.9391891956329346]